In [7]:
!pip install pandas openpyxl

  Using cached et_xmlfile-1.1.0-py3-none-any.whl.metadata (1.8 kB)
   ---------------------------------------- 0.0/250.9 kB ? eta -:--:--
   - -------------------------------------- 10.2/250.9 kB ? eta -:--:--
   ------ -------------------------------- 41.0/250.9 kB 653.6 kB/s eta 0:00:01
   ---------------------------------------- 250.9/250.9 kB 2.6 MB/s eta 0:00:00
Using cached et_xmlfile-1.1.0-py3-none-any.whl (4.7 kB)



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: C:\Users\sscho\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [13]:
import sqlite3
from datetime import datetime, timedelta
import pandas as pd

# Connect to the SQLite database
conn = sqlite3.connect('../highscores_backup.db')
cursor = conn.cursor()

# Function to convert string to datetime with None handling
def str_to_datetime(time_str):
    if time_str is None:
        return None
    return datetime.strptime(time_str, '%Y-%m-%d %H:%M:%S.%f')

# Query to get the daily scores and player information for all players and all days
cursor.execute("""
SELECT
    ds.score as sekker,
    ds.startedAt as start,
    ds.stoppedAt as slutt,
    ds.player_id,
    p.name,
    p.team,
    ds.date,
    CASE
        WHEN p.name LIKE '%(O18)' THEN 1
        ELSE 0
    END as is_over_18
FROM daily_scores ds
JOIN player p ON ds.player_id = p.id
""")
daily_scores = cursor.fetchall()

# Initialize an empty list to store results
all_results = []

# Process each score record
for score in daily_scores:
    presses_count = int(score[0])
    start_datetime = str_to_datetime(score[1])
    stop_datetime = str_to_datetime(score[2])
    player_id = score[3]
    name = score[4]
    team = score[5]
    date = score[6]
    is_over_18 = score[7]

    # Handle cases where start_datetime or stop_datetime is None
    if start_datetime is None or stop_datetime is None:
        tid_med_pause = None
        tid_uten_pause = None
        tid_per_sekk = None
    else:
        tid_med_pause = stop_datetime - start_datetime
        
        # Query to get the total break time for the day
        cursor.execute("""
        SELECT COALESCE(SUM(julianday(b.end_time) - julianday(b.start_time)), 0) AS break_time_total
        FROM breaks b
        WHERE b.start_time BETWEEN ? AND ? || ' 23:59:59'
        AND b.end_time BETWEEN ? AND ? || ' 23:59:59'
        """, (date, date, date, date))
        break_time_total = cursor.fetchone()[0]

        break_time_timedelta = timedelta(days=break_time_total)
        tid_uten_pause = tid_med_pause - break_time_timedelta
        tid_per_sekk = tid_uten_pause.total_seconds() / 86400 / presses_count if presses_count > 0 else None

    # Create a result dictionary, storing None values where applicable
    result = {
        'dag': date,
        'player_id': player_id,
        "navn": name,
        "team": team,
        "o18": is_over_18,
        'sekker': presses_count,
        "start": score[1],
        "slutt": score[2],
        'tid_med_pause': tid_med_pause.total_seconds() / 3600 / 24 if tid_med_pause else None,  # in days
        'pause': break_time_timedelta.total_seconds() / 3600 / 24 if start_datetime and stop_datetime else None,  # in days
        'tid_uten_pause': tid_uten_pause.total_seconds() / 3600 / 24 if tid_uten_pause else None,  # in days
        'tid_per_sekk': tid_per_sekk,
    }

    # Append the result to the list
    all_results.append(result)

# Convert the list of results into a DataFrame
df = pd.DataFrame(all_results)

# Save the DataFrame to an Excel file
filename = "result.xlsx"
df.to_excel(filename, index=False)

print(f"Data has been written to {filename}")


Data has been written to result.xlsx
